# Fine-tuning a model with the Trainer API

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


/Users/isab7070/miniconda3/envs/mynlp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Map: 100%|██████████| 1725/1725 [00:00<00:00, 10948.57 examples/s]


In [3]:
from transformers import TrainingArguments

training_args = TrainingArguments("test-trainer")

In [4]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [6]:
trainer.train()

 36%|███▋      | 500/1377 [23:16<40:22,  2.76s/it]  

{'loss': 0.5183, 'learning_rate': 3.184458968772695e-05, 'epoch': 1.09}


 73%|███████▎  | 1000/1377 [42:59<15:36,  2.48s/it] 

{'loss': 0.2844, 'learning_rate': 1.3689179375453886e-05, 'epoch': 2.18}


100%|██████████| 1377/1377 [56:54<00:00,  2.48s/it]

{'train_runtime': 3414.3723, 'train_samples_per_second': 3.223, 'train_steps_per_second': 0.403, 'train_loss': 0.3253657687982962, 'epoch': 3.0}


TrainOutput(global_step=1377, training_loss=0.3253657687982962, metrics={'train_runtime': 3414.3723, 'train_samples_per_second': 3.223, 'train_steps_per_second': 0.403, 'train_loss': 0.3253657687982962, 'epoch': 3.0})

In [7]:
predictions = trainer.predict(tokenized_datasets["validation"])
print(predictions.predictions.shape, predictions.label_ids.shape)

100%|██████████| 51/51 [00:26<00:00,  1.92it/s]

(408, 2) (408,)


In [8]:
import numpy as np

preds = np.argmax(predictions.predictions, axis=-1)

In [10]:
import evaluate

metric = evaluate.load("glue", "mrpc")
metric.compute(predictions=preds, references=predictions.label_ids)

{'accuracy': 0.8676470588235294, 'f1': 0.9075342465753423}

In [11]:
def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [12]:
training_args = TrainingArguments("test-trainer", evaluation_strategy="epoch")
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
trainer.train()

  0%|          | 0/1377 [00:00<?, ?it/s]

                                                    
 33%|███▎      | 459/1377 [16:22<23:36,  1.54s/it]

{'eval_loss': 0.36380577087402344, 'eval_accuracy': 0.8357843137254902, 'eval_f1': 0.8877721943048577, 'eval_runtime': 27.3163, 'eval_samples_per_second': 14.936, 'eval_steps_per_second': 1.867, 'epoch': 1.0}


 36%|███▋      | 500/1377 [17:44<27:03,  1.85s/it]  

{'loss': 0.4966, 'learning_rate': 3.184458968772695e-05, 'epoch': 1.09}


...

 73%|███████▎  | 1000/1377 [35:53<13:09,  2.09s/it] 

{'loss': 0.2802, 'learning_rate': 1.3689179375453886e-05, 'epoch': 2.18}


                                                   
100%|██████████| 1377/1377 [49:21<00:00,  2.15s/it]

{'eval_loss': 0.6796513795852661, 'eval_accuracy': 0.8651960784313726, 'eval_f1': 0.9036777583187391, 'eval_runtime': 29.2799, 'eval_samples_per_second': 13.934, 'eval_steps_per_second': 1.742, 'epoch': 3.0}
{'train_runtime': 2961.4387, 'train_samples_per_second': 3.716, 'train_steps_per_second': 0.465, 'train_loss': 0.3133214896540756, 'epoch': 3.0}


TrainOutput(global_step=1377, training_loss=0.3133214896540756, metrics={'train_runtime': 2961.4387, 'train_samples_per_second': 3.716, 'train_steps_per_second': 0.465, 'train_loss': 0.3133214896540756, 'epoch': 3.0})